In [ ]:
import math, random, copy, asyncio, time, tabulate, functools

In [ ]:
class Gene():
  def __init__(self, x, y):
    self.x = x
    self.y = y
  
  def euclidianDistance(self, individual):
    return math.sqrt(pow((individual.x - self.x),2) + pow((individual.y - self.y),2))

In [ ]:
class Individual():
  def __init__(self, genes):
    self.genes = genes

    self.calcFitness()

  def calcFitness(self):
    sum = 0
    
    for i in range(len(self.genes)-1):
      sum += self.genes[i].euclidianDistance(self.genes[i+1])

    sum += self.genes[-1].euclidianDistance(self.genes[0])

    self.fitness = sum
  
  def route(self):
    route = ''
    for gene in self.genes:
      route += f' ({gene.x}, {gene.y})'
    return route

In [ ]:
def geneticAlgorithm(src, crossRate = 0.8, mutRate = 0.1, popSize = 300, genNum = 50):
  population = []
  parents = []
  previousBestIndividual = None
  bestIndividual = None
  convCount = 0

  file = open(src, "r")
  data = file.readlines()
  file.close()
  
  genes = []

  for i in range(1, len(data)):
    coord = data[i].split(" ")
    genes.append(Gene(int(coord[0]), int(coord[1])))
  
  def avaliation():
    nonlocal bestIndividual
    nonlocal convCount
    previousBestIndividual = copy.deepcopy(bestIndividual)
    for i in range(len(population)):
      bestIndividual = copy.deepcopy(population[i]) if bestIndividual is None or population[i].fitness < bestIndividual.fitness else bestIndividual
    if not(previousBestIndividual is None) and bestIndividual.fitness >= previousBestIndividual.fitness:
      convCount+=1
    else:
      convCount = 0
      
  def generatePopulation():
    for i in range(popSize):
      population.append(Individual(random.sample(genes, len(genes))))

  def selection():
    nonlocal parents
    parents = []
    availableOptions = copy.copy(population)
    for i in range(popSize):
      if len(availableOptions) > 1:
        option1, option2 = random.sample(availableOptions, 2)
        if option1.fitness > option2.fitness:
          option1, option2 = option2, option1
        parents.append(option1)
        availableOptions.remove(option1)
      else:
        parents.append(availableOptions[0])

  def crossover():
    for i in range(0,len(parents), 2):
      numGenes = len(parents[i].genes)
      child1 = [0] * numGenes
      child2 = [0] * numGenes

      parent1 = parents[i].genes
      parent2 = parents[i+1].genes

      if random.random() <= crossRate:
        pt1, pt2 = random.sample(range(len(child1)), 2)
        
        if pt1 > pt2:
          pt1, pt2 = pt2, pt1
        
        for j in range(pt1, pt2):
          child1[j] = parent1[j]
          child2[j] = parent2[j]
        
        parent1rest = [item for item in parent1 if item not in child2]
        parent2rest = [item for item in parent2 if item not in child1]

        k = 0
        for j in range(0, numGenes):
          if j < pt1 or j >= pt2:
            child1[j] = parent2rest[k]
            child2[j] = parent1rest[k]
            k+=1

      else:
        child1 = copy.copy(parent1)
        child2 = copy.copy(parent2)

      population.append(Individual(child1))
      population.append(Individual(child2))
  
  def mutation():
    for i in range(len(population)):
      if random.random() <= mutRate:

        swap1, swap2 = random.sample(range(len(population[i].genes)), 2)
          
        gene1 = population[i].genes[swap1]
        gene2 = population[i].genes[swap2]
      
        population[i].genes[swap1] = gene2
        population[i].genes[swap2] = gene1

        population[i].calcFitness()

  def elitism():
    def rank(individual):
      return individual.fitness

    population.sort(key=rank)

    del population[-(popSize):]

      
  generatePopulation()

  genCount = 1

  while convCount != genNum:
    avaliation()
    selection()
    crossover()
    mutation()
    elitism()
    genCount+=1
  return bestIndividual, genCount

In [ ]:
def runGenAlgorithm(instance, runs = 10):
  results = []
  
  for i in range(runs):
    start = time.time()
    bestIndividual, numGenerations = geneticAlgorithm(src = instance)
    end = time.time()
    elapsed = time.strftime("%H:%M:%S", time.gmtime(end-start))
    results.append({
        'run': i+1,
        'bestFitness': bestIndividual.fitness,
        'numGenerations': numGenerations,
        'totalTime': elapsed,
        'bestRoute': bestIndividual.route(),
    })

  avgGen = functools.reduce(lambda x, y: x+y['numGenerations'], results, 0)/runs

  avg = functools.reduce(lambda x, y: x+y['bestFitness'], results, 0)/runs

  sdeviation = math.sqrt( sum(map(lambda x: pow((x['bestFitness'] - avg), 2), results))/runs )

  print(f'Instância: {instance}, fitness médio: {avg}, desvio-padrão: {sdeviation}, número médio gerações: {avgGen}')
  print('Detalhes:')
  table = list(map(lambda x: list(x.values()), results))
  headers = ["Execução", "Melhor Fitnes", "Número gerações", "Tempo", "Melhor Rota"]
  print(tabulate.tabulate(table, headers))

In [ ]:
runGenAlgorithm('tspcit30.dat')

Instância: tspcit30.dat, fitness médio: 52305.568392245616, desvio-padrão: 1548.9777996760083, número médio gerações: 157.3
Detalhes:
  Execução    Melhor Fitnes    Número gerações  Tempo     Melhor Rota
----------  ---------------  -----------------  --------  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
         1          52681.3                128  00:00:02  (4395, 8125) (3779, 8476) (3415, 8778) (2269, 9471) (4956, 9941) (6864, 8424) (8490, 9128) (9982, 9280) (7986, 5132) (6485, 4549) (5189, 3543) (6814, 3157) (6651, 2418) (7394, 2819) (8813, 1803) (8698, 307) (7588, 547) (6823, 1260) (2969, 318) (1020, 1900) (306, 1331) (450, 2089) (2832, 2642) (33

In [ ]:
runGenAlgorithm('tspcit100.dat')

Instância: tspcit100.dat, fitness médio: 32704.532500961137, desvio-padrão: 2520.4398904479563, número médio gerações: 940.3
Detalhes:
  Execução    Melhor Fitnes    Número gerações  Tempo     Melhor Rota
----------  ---------------  -----------------  --------  -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
runGenAlgorithm('tspcit101.dat')

Instância: tspcit101.dat, fitness médio: 93994.14892991757, desvio-padrão: 5652.419217519069, número médio gerações: 847.4
Detalhes:
  Execução    Melhor Fitnes    Número gerações  Tempo     Melhor Rota
----------  ---------------  -----------------  --------  -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------